In [ ]:
! unzip /content/final.zip

Archive:  /content/final.zip
   creating: final/
   creating: final/raw-documents/
  inflating: final/raw-documents/PT_01.txt  
  inflating: final/raw-documents/PT_02.txt  
  inflating: final/raw-documents/PT_03.txt  
  inflating: final/raw-documents/PT_04.txt  
  inflating: final/raw-documents/PT_05.txt  
  inflating: final/raw-documents/PT_06.txt  
  inflating: final/raw-documents/PT_07.txt  
  inflating: final/raw-documents/PT_08.txt  
  inflating: final/raw-documents/PT_09.txt  
  inflating: final/raw-documents/PT_10.txt  
  inflating: final/raw-documents/PT_100.txt  
  inflating: final/raw-documents/PT_101.txt  
  inflating: final/raw-documents/PT_102.txt  
  inflating: final/raw-documents/PT_103.txt  
  inflating: final/raw-documents/PT_104.txt  
  inflating: final/raw-documents/PT_105.txt  
  inflating: final/raw-documents/PT_106.txt  
  inflating: final/raw-documents/PT_107.txt  
  inflating: final/raw-documents/PT_108.txt  
  inflating: final/raw-documents/PT_109.txt  
  infla

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
file_path = "/content/final/subtask-2-annotations.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [ ]:
# Grouping by Document_ID and create lists of narratives
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)

In [ ]:
documents_folder = '/content/final/raw-documents'


def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            doc_id = filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

document_texts = load_documents(documents_folder)

df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df

Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,PT_01.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Ativistas climáticos ou idiotas úteis?\n\n“Pel...
1,PT_02.txt,"[URW: Discrediting the West, Diplomacy, URW: B...","[URW: Discrediting the West, Diplomacy: Other,...",Rússia: posição dos EUA sobre o ataque à Câmar...
2,PT_03.txt,"[URW: Praise of Russia, URW: Discrediting Ukra...",[URW: Discrediting Ukraine: Discrediting Ukrai...,Pelo menos 20 mercenários polacos foram mortos...
3,PT_04.txt,[URW: Discrediting Ukraine],[URW: Discrediting Ukraine: Discrediting Ukrai...,Como é que notório terrorista do Daesh consegu...
4,PT_05.txt,[URW: Blaming the war on others rather than th...,[URW: Blaming the war on others rather than th...,Bundestag vota a favor do projeto de resolução...
...,...,...,...,...
395,PT_URW_414.txt,"[URW: Discrediting the West, Diplomacy, URW: B...",[URW: Discrediting Ukraine: Discrediting Ukrai...,"Armas nucleares na Ucrânia, a responsabilidade..."
396,PT_URW_417.txt,[URW: Blaming the war on others rather than th...,[URW: Amplifying war-related fears: NATO shoul...,"""Tinha dito que ia chegar o momento em que Zel..."
397,PT_URW_418.txt,[CC: Criticism of institutions and authorities],[CC: Criticism of institutions and authorities...,OE2025: Climáximo critica orçamento que diz ga...
398,PT_URW_419.txt,"[URW: Praise of Russia, URW: Russia is the Vic...",[URW: Praise of Russia: Russia is a guarantor ...,Fórum em Ashkhabad. Putin reitera que nova ord...


In [ ]:
# Converting the 'High_Level_Narratives_List' column from string to list
df['High_Level_Narratives_List'] = df['High_Level_Narratives_List'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Exploding the dataframe to have one row per High Level Narrative
df = df.explode('High_Level_Narratives_List')

df = df.rename(columns={"High_Level_Narratives_List": "High_Level_Narrative"})

df

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_01.txt,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,Ativistas climáticos ou idiotas úteis?\n\n“Pel...
1,PT_02.txt,"URW: Discrediting the West, Diplomacy","[URW: Discrediting the West, Diplomacy: Other,...",Rússia: posição dos EUA sobre o ataque à Câmar...
1,PT_02.txt,URW: Blaming the war on others rather than the...,"[URW: Discrediting the West, Diplomacy: Other,...",Rússia: posição dos EUA sobre o ataque à Câmar...
1,PT_02.txt,URW: Discrediting Ukraine,"[URW: Discrediting the West, Diplomacy: Other,...",Rússia: posição dos EUA sobre o ataque à Câmar...
2,PT_03.txt,URW: Praise of Russia,[URW: Discrediting Ukraine: Discrediting Ukrai...,Pelo menos 20 mercenários polacos foram mortos...
...,...,...,...,...
398,PT_URW_419.txt,URW: Russia is the Victim,[URW: Praise of Russia: Russia is a guarantor ...,Fórum em Ashkhabad. Putin reitera que nova ord...
399,PT_URW_420.txt,"URW: Discrediting the West, Diplomacy","[URW: Discrediting the West, Diplomacy: Other,...",Rússia defende legitimidade do lançamento de m...
399,PT_URW_420.txt,URW: Praise of Russia,"[URW: Discrediting the West, Diplomacy: Other,...",Rússia defende legitimidade do lançamento de m...
399,PT_URW_420.txt,URW: Russia is the Victim,"[URW: Discrediting the West, Diplomacy: Other,...",Rússia defende legitimidade do lançamento de m...


In [ ]:
df = df[df["High_Level_Narrative"].str.startswith("CC")].reset_index(drop=True)
df.head()

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_01.txt,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,Ativistas climáticos ou idiotas úteis?\n\n“Pel...
1,PT_100.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,Estudo da NASA indica que o Nordeste e grande ...
2,PT_101.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,Humanidade sofre com 'epidemia de calor extrem...
3,PT_117.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,ONU lança novas orientações para lidar com inc...
4,PT_134.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,"Alterações climáticas podem matar até 14,5 mil..."


In [ ]:
print(f'{len(df)} CC High Level Narratives after spliting multiple URW High Level Narratives in multiple rows')

We have 250 CC High Level Narratives after spliting multiple URW High Level Narratives in multiple rows


In [ ]:
# Here we will add high level narrative to text with a | seperating high level narrative and text
df["Text"] = df["High_Level_Narrative"] + " | " + df["Text"]

df.head()

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_01.txt,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,CC: Hidden plots by secret schemes of powerful...
1,PT_100.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Estudo da NASA ...
2,PT_101.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Humanidade sofr...
3,PT_117.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | ONU lança novas...
4,PT_134.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Alterações clim...


In [ ]:
import ast  # Safer alternative to eval()

def filter_sub_narratives(row):
    high_level = row["High_Level_Narrative"].strip()

    if isinstance(row["Sub_Narratives_List"], str):
        sub_narratives = ast.literal_eval(row["Sub_Narratives_List"])
    else:
        sub_narratives = row["Sub_Narratives_List"]

    # Keeping only sub-narratives that start with the high-level narrative
    filtered = [sub for sub in sub_narratives if sub.startswith(high_level)]
    return filtered

# Applying filtering function
df["Filtered_Sub_Narratives"] = df.apply(filter_sub_narratives, axis=1)
df['Filtered_Sub_Narratives']

,Filtered_Sub_Narratives
0,[CC: Hidden plots by secret schemes of powerfu...
1,[CC: Amplifying Climate Fears: Amplifying exis...
2,[CC: Amplifying Climate Fears: Amplifying exis...
3,[CC: Amplifying Climate Fears: Amplifying exis...
4,[CC: Amplifying Climate Fears: Amplifying exis...
...,...
245,[CC: Amplifying Climate Fears: Other]
246,[CC: Downplaying climate change: Other]
247,[CC: Criticism of climate movement: Climate mo...
248,[CC: Questioning the measurements and science:...


In [ ]:
# Unique sub-narratives
sub_narratives = list(set(label for sublist in df["Filtered_Sub_Narratives"] for label in sublist))
len(sub_narratives)

34

In [ ]:
def preprocess_multi_label(data, narratives):
    label_vectors = []
    for narratives_list in data['Filtered_Sub_Narratives']:
        vector = [1 if narrative in narratives_list else 0 for narrative in narratives]
        label_vectors.append(vector)
    data['Labels'] = label_vectors
    return data

# Encoding sub-level narratives
df = preprocess_multi_label(df, sub_narratives)
df.head()

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text,Filtered_Sub_Narratives,Labels
0,PT_01.txt,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,PT_100.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Estudo da NASA ...,[CC: Amplifying Climate Fears: Amplifying exis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,PT_101.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Humanidade sofr...,[CC: Amplifying Climate Fears: Amplifying exis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,PT_117.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | ONU lança novas...,[CC: Amplifying Climate Fears: Amplifying exis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,PT_134.txt,CC: Amplifying Climate Fears,[CC: Amplifying Climate Fears: Amplifying exis...,CC: Amplifying Climate Fears | Alterações clim...,[CC: Amplifying Climate Fears: Amplifying exis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [ ]:
! pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 6.8 MB/s eta 0:00:00


In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np

def multilabel_train_test_split(X, Y, test_size=0.2):
    """
    Uses iterative stratification to split multi-label data while preserving label distributions.

    Parameters:
    - X: List or NumPy array (text data)
    - Y: NumPy array (multi-label format)
    - test_size: Fraction of test data

    Returns:
    - X_train, X_test, Y_train, Y_test
    """
    X = np.array(X).reshape(-1, 1)
    Y = np.array(Y)

    X_train, Y_train, X_test, Y_test = iterative_train_test_split(X, Y, test_size)

    return X_train.ravel(), X_test.ravel(), Y_train, Y_test

labels = np.array(df['Labels'].tolist(), dtype=np.float32)

indices = np.arange(len(df))

train_indices, train_labels, test_indices, test_labels = iterative_train_test_split(indices.reshape(-1, 1), labels, test_size=0.2)

train_indices = train_indices.ravel()
test_indices = test_indices.ravel()

# Extracting corresponding texts using indices
train_texts = df.iloc[train_indices]["Text"].tolist()
test_texts = df.iloc[test_indices]["Text"].tolist()



In [ ]:
import random

def handle_rare_classes(data, labels, min_samples=15, max_duplication=3):
    """
    Handles rare classes by undersampling majority classes and applying controlled oversampling
    with minor augmentations to prevent overfitting.

    - `min_samples`: Minimum number of instances per class after processing.
    - `max_duplication`: Maximum times a rare sample is duplicated.
    """
    label_sums = np.sum(labels, axis=0)
    rare_classes = np.where(label_sums < min_samples)[0]

    if len(rare_classes) == 0:
        print("No rare classes found. Skipping balancing.")
        return data, labels

    print(f"Rare classes found: {len(rare_classes)}. Applying undersampling & controlled oversampling...")

    data_df = pd.DataFrame({"Text": data})

    # 1. Undersample Majority Classes
    label_counts = np.sum(labels, axis=0)
    majority_classes = np.where(label_counts > min_samples * 5)[0]

    if len(majority_classes) > 0:
        print(f"Undersampling {len(majority_classes)} majority classes...")
        drop_indices = []
        for maj_class in majority_classes:
            maj_indices = [i for i in range(len(labels)) if labels[i][maj_class] == 1]
            keep_size = min(len(maj_indices), min_samples * 5)
            drop_indices.extend(random.sample(maj_indices, len(maj_indices) - keep_size))

        # Drop excess samples
        data_df.drop(drop_indices, inplace=True)
        labels = np.delete(labels, drop_indices, axis=0)
        data_df.reset_index(drop=True, inplace=True)

    # 2. Controlled Oversampling for Rare Classes
    new_texts = []
    new_labels = []

    for rare_class in rare_classes:
        rare_indices = [i for i in range(len(labels)) if labels[i][rare_class] == 1]

        if len(rare_indices) == 0:
            continue

        num_to_add = min_samples - len(rare_indices)
        num_to_add = min(num_to_add, max_duplication * len(rare_indices))

        for _ in range(num_to_add):
            idx = random.choice(rare_indices)
            new_text = data_df.iloc[idx]["Text"]

            # Minor Text Augmentation to Prevent Overfitting
            if len(new_text) > 20:
                words = new_text.split()
                if len(words) > 5:
                    random.shuffle(words[:5])
                new_text = " ".join(words)

            new_texts.append(new_text)
            new_labels.append(labels[idx])

    # Add new samples to dataset
    if len(new_texts) > 0:
        new_data_df = pd.DataFrame({"Text": new_texts})
        data_df = pd.concat([data_df, new_data_df], ignore_index=True)
        labels = np.vstack([labels, np.array(new_labels)])

    print(f"Final dataset size: {len(data_df)} samples")

    return data_df["Text"], labels.astype(np.float32)

In [ ]:
import numpy as np

def print_label_distribution(labels):
    """Prints the count of positive instances for each class."""
    label_counts = np.sum(labels, axis=0)
    print("Label Counts:", label_counts)
    print("Min:", label_counts.min(), "Max:", label_counts.max())

# Checking class distribution after applying handle_rare_classes
print("Before balancing:")
print_label_distribution(np.array(train_labels))

train_texts, train_labels = handle_rare_classes(train_texts, train_labels, min_samples=15, max_duplication=3)

print("After balancing:")
print_label_distribution(np.array(train_labels))


Before balancing:
Label Counts: [ 2.  1.  7.  2. 29.  8.  1.  1.  5.  2. 71.  2. 19.  2.  1.  7.  3.  3.
  1.  1.  1.  2. 25. 50.  6.  1.  1.  9.  8. 10.  6.  1.  1.  9.]
Min: 1.0 Max: 71.0
Rare classes found: 29. Applying undersampling & controlled oversampling...
Final dataset size: 363 samples
After balancing:
Label Counts: [10.  4. 26. 13. 35. 17.  4.  8. 21.  8. 82. 12. 24.  8.  4. 15. 13. 12.
  7.  4.  4. 11. 29. 60. 23.  8.  4. 25. 17. 16. 16.  4.  4. 20.]
Min: 4.0 Max: 82.0


In [ ]:
from datasets import Dataset, DatasetDict

# Converting to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "labels": [np.array(lbl, dtype=np.float32).tolist() for lbl in train_labels]
})

test_dataset = Dataset.from_dict({
    "text": test_texts,
    "labels": [np.array(lbl, dtype=np.float32).tolist() for lbl in test_labels]
})

datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Use XLM-RoBERTa which handles multiple languages including Portuguese
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(sub_narratives),
    problem_type="multi_label_classification"
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import EvalPrediction

def find_best_threshold(preds, labels):
    """Find the best threshold for F1-score optimization."""
    best_f1 = 0
    best_threshold = 0.5  # Default threshold
    thresholds = np.arange(0.1, 0.9, 0.05)  # Try thresholds from 0.1 to 0.9

    for threshold in thresholds:
        binarized_preds = (preds > threshold).astype(int)
        f1 = f1_score(labels, binarized_preds, average="weighted", zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

def compute_metrics(p: EvalPrediction):
    """Compute F1, Precision, Recall with optimal threshold selection."""
    sigmoid_preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()

    # Ground-truth labels
    labels = p.label_ids

    best_threshold = find_best_threshold(sigmoid_preds, labels)

    preds = (sigmoid_preds > best_threshold).astype(int)

    # metrics
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)

    return {
        "best_threshold": best_threshold,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    learning_rate=6e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    seed=42,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-57-763004834412>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Best Threshold,Precision,Recall,F1
1,No log,0.068994,0.200000,0.543150,0.915493,0.661295
2,0.066000,0.070008,0.200000,0.576543,0.873239,0.676351
3,0.058000,0.071297,0.150000,0.620746,0.788732,0.669022
4,0.054100,0.064965,0.150000,0.555978,0.915493,0.677998
5,0.046500,0.065789,0.250000,0.592478,0.830986,0.679650
6,0.046700,0.062537,0.200000,0.576635,0.873239,0.685699
7,0.040600,0.066231,0.100000,0.555296,0.915493,0.670952
8,0.034100,0.066113,0.150000,0.618852,0.887324,0.712932
9,0.029500,0.072613,0.200000,0.572229,0.788732,0.658230
10,0.028100,0.067854,0.150000,0.599676,0.901408,0.711448


In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

raw_predictions = trainer.predict(tokenized_datasets["test"])

sigmoid_preds = torch.sigmoid(torch.tensor(raw_predictions.predictions)).numpy()

labels = np.array(test_labels)
best_threshold = find_best_threshold(sigmoid_preds, labels)

binary_predictions = (sigmoid_preds > best_threshold).astype(int)

precision = precision_score(labels, binary_predictions, average="weighted", zero_division=0)
recall = recall_score(labels, binary_predictions, average="weighted", zero_division=0)
f1 = f1_score(labels, binary_predictions, average="weighted", zero_division=0)

test_df = df.iloc[test_indices].copy()
test_df["Predicted Labels"] = binary_predictions.tolist()  # predictions
test_df["Actual Labels"] = test_labels.tolist()  # actual labels
test_df["Threshold Used"] = best_threshold  # threshold for reference



# Evaluation Metrics
evaluation_metrics = {
    "best_threshold": best_threshold,
    "precision": precision,
    "recall": recall,
    "f1": f1
}
evaluation_metrics

{'best_threshold': 0.25000000000000006,
 'precision': 0.626280409731114,
 'recall': 0.8028169014084507,
 'f1': 0.6874552125587752}

In [ ]:
# Function to map binary predictions to actual narrative labels
def convert_binary_predictions_to_labels(binary_predictions, label_names):
    """
    Converts binary predictions (0/1 array) into a list of predicted sub-narrative labels.

    Parameters:
    - binary_predictions: List of binary predictions (0/1)
    - label_names: List of all possible label names corresponding to each index

    Returns:
    - List of predicted labels for each row
    """
    return [[label_names[i] for i, pred in enumerate(pred_row) if pred == 1] for pred_row in binary_predictions]

unique_labels = df["Filtered_Sub_Narratives"].explode().unique().tolist()

test_df["Predicted Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Predicted Labels"], unique_labels)

test_df["Actual Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Actual Labels"], unique_labels)

test_df=test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]]

test_df.to_csv("results_CC_sub.csv", index=False)

In [ ]:
test_df_preview = test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]].head(10)
test_df_preview['Actual Sub_Narratives'][14]

['CC: Questioning the measurements and science: Methodologies/metrics used are unreliable/faulty',
 'CC: Hidden plots by secret schemes of powerful groups: Other']

In [ ]:
!zip -r PT-expr-2.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-1365/ (stored 0%)
  adding: content/results/checkpoint-1365/sentencepiece.bpe.model (deflated 49%)
  adding: content/results/checkpoint-1365/training_args.bin (deflated 52%)
  adding: content/results/checkpoint-1365/config.json (deflated 67%)
  adding: content/results/checkpoint-1365/special_tokens_map.json (deflated 52%)
  adding: content/results/checkpoint-1365/tokenizer.json (deflated 76%)
  adding: content/results/checkpoint-1365/tokenizer_config.json (deflated 76%)
  adding: content/results/checkpoint-1365/rng_state.pth (deflated 25%)
  adding: content/results/checkpoint-1365/model.safetensors (deflated 31%)
  adding: content/results/checkpoint-1365/trainer_state.json (deflated 77%)
  adding: content/results/checkpoint-1365/scheduler.pt (deflated 56%)
  adding: content/results/checkpoint-1365/optimizer.pt (deflated 69%)
